# Mision To Mars

In [52]:
%reload_ext lab_black

In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re

# NASA Mars News

In [54]:
# Define NASA News url
NASA_URL = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

driver = webdriver.Firefox()
driver.get(NASA_URL)
driver.implicitly_wait(10)
nasa_html = driver.page_source

# Loading page
nasa_soup = BeautifulSoup(nasa_html, "lxml")

In [55]:
# Extract News Title
news_title = [
    tag.text
    for tag in [
        li for ul in nasa_soup for li in ul.findAll("div", class_="content_title")[0]
    ]
]

# Extract News Paragraph
news_p = [
    tag
    for tag in [
        li
        for ul in nasa_soup
        for li in ul.findAll("div", class_="article_teaser_body")[0]
    ]
]

driver.close()

# Feature image

In [56]:
# Define Feature Image Url
IMAGE_URL = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

# Loading page
response = requests.get(IMAGE_URL)
image_url_soup = BeautifulSoup(response.text, "html.parser")

image_url = image_url_soup.find("article", {"class": "carousel_item"})["style"]

featured_image_url = re.findall("url\((.*?)\)", image_url)[0].replace("'", "")

featured_image_url = "https://www.jpl.nasa.gov" + featured_image_url

# featured_image_url

In [57]:
# Create the result list
nasa_result = []
nasa_result.append(news_title)
nasa_result.append(news_p)
nasa_result.append(featured_image_url)
nasa_result

[['The MarCO Mission Comes to an End'],
 ['The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'],
 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17172-1920x1200.jpg']

# Twitter Mars Weather

In [58]:
# Define Twitter Mars Report url
MARS_WEATHER_URL = "https://twitter.com/marswxreport?lang=en"

# Loading page
response = requests.get(MARS_WEATHER_URL)
mars_weather_soup = BeautifulSoup(response.text, "html.parser")

In [59]:
# Finding last tweet
mars_weather_soup.find("a", {"class": "twitter-timeline-link u-hidden"}).decompose()
tweet = mars_weather_soup.find("div", class_="js-tweet-text-container").text

tweet

'\nInSight sol 447 (2020-02-28) low -92.6ºC (-134.6ºF) high -11.1ºC (12.0ºF)\nwinds from the SSW at 5.8 m/s (13.1 mph) gusting to 20.2 m/s (45.2 mph)\npressure at 6.30 hPa\n'

# Mars Space Facts

In [60]:
# Define Space Facts url
MARS_FACTS_URL = "https://space-facts.com/mars/"

# Loading page
response = requests.get(MARS_FACTS_URL)
mars_facts_soup = BeautifulSoup(response.text, "html.parser")

In [61]:
# Getting the facts about Mars
facts = f'{mars_facts_soup.find("table", class_="tablepress tablepress-id-p-mars")}'
facts

'<table class="tablepress tablepress-id-p-mars" id="tablepress-p-mars"><tbody><tr class="row-1 odd"><td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/></td></tr><tr class="row-2 even"><td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/></td></tr><tr class="row-3 odd"><td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.39 × 10^23 kg<br/> (0.11 Earths)</td></tr><tr class="row-4 even"><td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/moons/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/moons/deimos/">Deimos</a>)</td></tr><tr class="row-5 odd"><td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km<br/> (1.38 AU)</td></tr><tr class="row-6 even"><td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/></td></tr><tr class="row-7 odd"><td cl

# Mars Hemispheres

In [62]:
# Define Astrogeology url
MARS_HEM_URL = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

driver = webdriver.Firefox()
driver.get(MARS_HEM_URL)
driver.implicitly_wait(10)

# Create and empty list
hem_list = []

# Find all the links
links = driver.find_elements_by_class_name("thumb")

# Loop through the links, click on each and extract title and image url into a list of dicts
for l in range(len(links)):
    driver.find_elements_by_class_name("thumb")[l].click()

    title = driver.find_element_by_class_name("title")
    image_url = driver.find_element_by_link_text("Sample")

    hem_list.append({"title": title.text, "img_url": image_url.get_attribute("href")})
    driver.back()

print(hem_list)
driver.close()

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
